# Proyecto chatbot para Educacion

Se propone el desarrollo de una prueba de concepto de un chatbot educativo que utilice la metodología RAG (Retrieval Augmented Generation) para responder de manera precisa a preguntas sobre el contenido de un curso universitario. Este chatbot, alimentado con las notas de clase de un curso existente, funcionará como un tutor virtual personalizado, disponible las 24 horas del día para los estudiantes. La iniciativa tiene como objetivo principal mejorar la experiencia de aprendizaje al proporcionar un recurso adicional para consultar el conocimiento contenido en los apuntes elaborados por los docentes. El chatbot será capaz de comprender preguntas complejas, encontrar la información relevante en las notas de clase y generar respuestas coherentes y concisas. Se utilizarán redes neuronales preentrenadas, de acceso abierto y alojadas de forma local en el servidor de la facultad. Para el desarrollo de la prueba de concepto se utilizará el lenguaje Python. El proyecto abarca desde la recopilación y procesamiento de las notas de clase en formato PDF, Word u otro formato similar, hasta el desarrollo de una interfaz conversacional intuitiva mediante la librería Streamlit o similar.
Se espera que al utilizar el chatbot la consulta de los apuntes de clase por parte de los estudiantes aumente, de modo que se logre un aprendizaje más personalizado, una mayor comprensión de los conceptos y un ahorro de tiempo para los estudiantes. Además, se espera que este chatbot sea una herramienta valiosa para los docentes, al proporcionarles información sobre las áreas en las que los estudiantes tienen más interés o dificultades.

## Setup

In [1]:
import os
import numpy as np
#Retrieval libraries
from unidecode import unidecode
import re
import torch
from typing import List, Dict, Any
from langchain.schema import Document
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
#from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.retrievers import ContextualCompressionRetriever
#from langchain_community.document_compressors import SentenceTransformerRerank
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers.document_compressors import EmbeddingsFilter # <-- Importar
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

#LLM libraries
from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

In [5]:
from dotenv import load_dotenv

def get_api_key():
    """
    Loads the LLM API key from the .env file and returns it.
    
    Raises:
        ValueError: If the LLM_API_KEY is not found in the environment.
        
    Returns:
        str: The LLM API key.
    """
    # This line loads the environment variables from the .env file
    load_dotenv()
    
    # os.getenv() retrieves the value of the environment variable
    api_key = os.getenv("GOOGLE_API_KEY")
    
    # This is a critical security and robustness check
    if not api_key:
        raise ValueError("API Key not found. Make sure you have a .env file with LLM_API_KEY defined.")
        
    return api_key

#####################################
try:
    my_llm_key = get_api_key()
        
        # Now you can use the key in your application
    print("Successfully loaded API Key.")
        # For security, we only show the first and last few characters
    print(f"Key starts with: {my_llm_key[:4]}... and ends with: ...{my_llm_key[-4:]}")
        
        # Example of using the key with a fictional API call
        # some_llm_library.authenticate(api_key=my_llm_key)
        
except ValueError as e:
    print(f"Error: {e}")
        
os.environ["GOOGLE_API_KEY"] = my_llm_key
######################################

Successfully loaded API Key.
Key starts with: AIza... and ends with: ...CoTw


In [7]:
# --- Configuration ---
# Chequeo dinámico de dispositivo (CPU o GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

# Use a specific folder to store the database, models, etc.
PERSIST_DIRECTORY = "./db_chroma"
MODEL_CACHE_DIR = "./model_cache"
SOURCE_DOCS_PATH = "./knowledgeBase" 

# Define los parámetros de tu modelo de embedding
model_name = 'BAAI/bge-m3'
# Multi-Functionality: It can simultaneously perform the three common retrieval functionalities of embedding model: dense retrieval, multi-vector retrieval, and sparse retrieval.
# Multi-Linguality: It can support more than 100 working languages.
# Multi-Granularity: It is able to process inputs of different granularities, spanning from short sentences to long documents of up to 8192 tokens.

model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': True}

# Crea la instancia del embedding encoder
embedding_encoder = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    cache_folder=MODEL_CACHE_DIR
)
retrieval_reranker=HuggingFaceCrossEncoder(
    model_name="BAAI/bge-reranker-v2-m3"
    #"BAAI/bge-reranker-v2-m3"
    #"BAAI/bge-reranker-v2-minicpm-layerwise"
    #"Alibaba-NLP/gte-multilingual-reranker-base" #General Text Embedding"
)

Usando dispositivo: cpu


## Funciones auxiliares

In [8]:
def process_markdown_document(file_path):
    """Loads a Markdown document and returns a LangChain Document object.

    Args:
        file_path (str): The path to the Markdown file.

    Returns:
        langchain.document_loaders.TextLoader: A TextLoader object containing the document.
    """

    loader = TextLoader(file_path,encoding="UTF8")
    documents = loader.load()

    # Split documents by Markdown sections
    headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
    ]
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=True)
    md_header_splits = markdown_splitter.split_text(documents[0].page_content)
    return md_header_splits

# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )
    
def EmbeddDocsAndPersist(all_splits,embedding_encoder,PERSIST_DIRECTORY):
    # Crear embeddings y persistir la DB en disco
    # Esto solo se hace una vez o cuando los documentos cambian
    print("Creando y persistiendo la base de datos de vectores...")
    vectorStore = Chroma.from_documents(
        documents=all_splits,
        embedding=embedding_encoder,
        persist_directory=PERSIST_DIRECTORY
    )
    print("Base de datos creada y guardada.")
    return vectorStore

def load_persisted_db(embedding_encoder,PERSIST_DIRECTORY):
    # Cargar la DB desde el disco
    print("Cargando base de datos persistente...")
    vectorStore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embedding_encoder
    )
    print("Base de datos cargada.")
    return vectorStore

#Define a function for joining retrieved chunks
def join_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

## Inicilizacion de base de datos

In [9]:
all_docs = [] # <--- Lista para acumular todos los documentos

for filename in os.listdir(SOURCE_DOCS_PATH): # <--- Iteramos sobre los archivos
    if filename.endswith(".md"): # <--- Filtramos solo archivos .md
        file_path = os.path.join(SOURCE_DOCS_PATH, filename) # <--- Construimos la ruta completa
        processed_document_chunks = process_markdown_document(file_path) # <--- Procesamos cada archivo
        all_docs.extend(processed_document_chunks) # <--- Agregamos los documentos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=100,
    separators=["\n\n", "\n"])

chunked_splits = text_splitter.split_documents(all_docs)

# Add metadata
for i, doc in enumerate(chunked_splits):
    doc.metadata["doc_id"] = f"chunk_{i}"

In [10]:
#Codificacion de los chunks y Creacion de la base de datos
if len(os.listdir(PERSIST_DIRECTORY))==0:
    vectorStore = EmbeddDocsAndPersist(chunked_splits,embedding_encoder,PERSIST_DIRECTORY)
else:
    vectorStore = load_persisted_db(embedding_encoder,PERSIST_DIRECTORY)

Cargando base de datos persistente...


/tmp/ipykernel_293496/2031765100.py:48: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorStore = Chroma(


Base de datos cargada.


In [11]:
#Calculo tokens por chunk para ver si se sobrecarga el LLM elegido para embedding
from transformers import AutoTokenizer

# Load a pre-trained tokenizer (e.g., for BERT-base-uncased). BAAI/bge-m3
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text = chunked_splits[0].page_content
token_ids = tokenizer.encode(text)

# The number of tokens is the length of the token_ids list
num_tokens = len(token_ids)

print(f"Original text: '{text}'")
print(f"Number of tokens: {num_tokens}")

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Original text: 'Un problema de **Valores y Vectores Propios** consiste en encontrar los vectores \(v\) tales que son direcciones invariantes de la transformación lineal dada por la matriz \(A\) (NxN). Esto se expresa como: \(A v = \lambda v \) siendo \(\lambda\) el escalar que cambia el módulo del vector cuya dirección permanece invariante. Se denomina **autovalor** \( \lambda \) y **autovector** \(v\). El sistema de ecuaciones puede escribirse de la forma: \((A - \lambda I) v = 0 \) donde interesan las soluciones \(v\) distintas de la trivial,\(v = 0\). Esto está garantizado sí y sólo sí \(det(A - \lambda I) = 0\). El determinante constituye un polinomio de grado N en el autovalor \(\lambda \), y se denomina **polinomio característico**. Las raíces de dicho polinomio son los autovalores \(\lambda \) de la matriz \(A\) para los cuales existen los autovectores o direcciones invariantes \(v\). Por cada valor propio existe al menos una dirección invariante dada por el autovector \(v\).'
N

## Testing Fase Retrieval

### Armado de dataset de validacion

In [11]:
#Observamos chunks para armar evaluation_dataset
for i, doc in enumerate(chunked_splits):
    print(doc)

page_content='Un problema de **Valores y Vectores Propios** consiste en encontrar los vectores \(v\) tales que son direcciones invariantes de la transformación lineal dada por la matriz \(A\) (NxN). Esto se expresa como: \(A v = \lambda v \) siendo \(\lambda\) el escalar que cambia el módulo del vector cuya dirección permanece invariante. Se denomina **autovalor** \( \lambda \) y **autovector** \(v\). El sistema de ecuaciones puede escribirse de la forma: \((A - \lambda I) v = 0 \) donde interesan las soluciones \(v\) distintas de la trivial,\(v = 0\). Esto está garantizado sí y sólo sí \(det(A - \lambda I) = 0\). El determinante constituye un polinomio de grado N en el autovalor \(\lambda \), y se denomina **polinomio característico**. Las raíces de dicho polinomio son los autovalores \(\lambda \) de la matriz \(A\) para los cuales existen los autovectores o direcciones invariantes \(v\). Por cada valor propio existe al menos una dirección invariante dada por el autovector \(v\).' met

In [12]:
#Ejemplo de resultado para guiar la IA generativa
evaluation_dataset = [
    {
        "question": "¿Cuál es la condición de convergencia para el método de la potencia?",
        "ground_truth_doc_id": "chunk_3" # The ID of the paragraph with the answer
    },
    {
        "question": "¿Cuál es el pseudocodigo del método de la potencia?",
        "ground_truth_doc_id": "chunk_4"
    },
    
    #COMPLETAR CASOS
]

> Usamos IA generativa externa para crear casos de validacion a partir de los chunks. 

In [13]:
evaluation_dataset_AIgenerated = [
    {
        "question": "¿Qué es un autovector y un autovalor?",
        "ground_truth_doc_id": "chunk_0"
    },
    {
        "question": "¿Qué es el polinomio característico y cómo se relaciona con los autovalores?",
        "ground_truth_doc_id": "chunk_0"
    },
    {
        "question": "Menciona las tres categorías de métodos para la determinación de valores y vectores característicos.",
        "ground_truth_doc_id": "chunk_1" 
    },
    {
        "question": "¿En qué consisten los métodos iterativos para encontrar autovalores y cuál es el ejemplo tratado en el curso?",
        "ground_truth_doc_id": "chunk_2"
    },
    {
        "question": "¿Cuál es la condición de convergencia para el método de la potencia directa?",
        "ground_truth_doc_id": "chunk_3"
    },
    {
        "question": "¿Por qué es necesario usar escalamiento en el método de la potencia y cómo se realiza?",
        "ground_truth_doc_id": "chunk_3"
    },
    {
        "question": "¿Cuál es el pseudocodigo del método de la potencia?",
        "ground_truth_doc_id": "chunk_4"
    },
    {
        "question": "¿Cómo funciona el método de la potencia inversa y a qué autovalor de la matriz original converge?",
        "ground_truth_doc_id": "chunk_5"
    },
    {
        "question": "¿Qué es la deflación y para qué se utiliza en el cálculo de autovectores?",
        "ground_truth_doc_id": "chunk_6"
    },
    {
        "question": "¿Qué es la cuadratura y el error de truncamiento en la integración numérica?",
        "ground_truth_doc_id": "chunk_9"
    },
    {
        "question": "¿Cuál es la diferencia fundamental entre la cuadratura de Newton-Cotes y la de Gauss-Legendre?",
        "ground_truth_doc_id": "chunk_11" # This chunk explicitly contrasts with Newton-Cotes described in chunk_10
    },
    {
        "question": "¿Qué es la regla de los trapecios y cuál es su orden de exactitud?",
        "ground_truth_doc_id": "chunk_13"
    },
    {
        "question": "¿Cuál es el orden del error en la regla de los trapecios simple?",
        "ground_truth_doc_id": "chunk_15"
    },
    {
        "question": "¿Cómo cambia el orden del error al pasar de la regla de trapecios simple a la múltiple?",
        "ground_truth_doc_id": "chunk_18"
    },
    {
        "question": "¿Cuál es la fórmula de la regla de Simpson simple y cuántos puntos utiliza?",
        "ground_truth_doc_id": "chunk_21"
    },
    {
        "question": "¿Cuál es el orden del error para la regla de Simpson simple?",
        "ground_truth_doc_id": "chunk_23"
    },
    {
        "question": "¿Cuál es la fórmula de la regla de Simpson compuesta y cuál es el orden de su error?",
        "ground_truth_doc_id": "chunk_24"
    },
    {
        "question": "Describe la regla de cuadratura de Gauss de dos puntos, incluyendo los valores de las abscisas y los coeficientes.",
        "ground_truth_doc_id": "chunk_26"
    },
    {
        "question": "¿Qué es la extrapolación de Richardson y cuál es su propósito?",
        "ground_truth_doc_id": "chunk_28"
    },
    {
        "question": "¿En qué consiste la integración de Romberg y qué técnica aplica sucesivamente?",
        "ground_truth_doc_id": "chunk_29"
    }
]

### Testing vectorstore as retriever

In [14]:
def evaluate_vectorstore_as_retriever(eval_dataset, vector_store, k=5):
    """
    Evaluates the performance of a retriever using a given dataset.

    Args:
        eval_dataset (list): A list of dictionaries with "question" and "ground_truth_doc_id".
        vector_store: The ChromaDB vector store instance.
        k (int): The number of top documents to retrieve for evaluation.

    Returns:
        dict: A dictionary containing the calculated metrics.
    """
    hits = 0
    reciprocal_ranks = []
    misses = [] # To store information about failed queries for later analysis

    print(f"Starting evaluation for k={k}...")

    for item in eval_dataset:
        question = item["question"]
        ground_truth_id = item["ground_truth_doc_id"]
        
        # Perform the similarity search
        # The result is a list of tuples: [(Document, score), (Document, score), ...]
        retrieved_docs_with_scores = vector_store.similarity_search_with_score(question, k=k)
        
        # Extract the doc_ids from the metadata of the retrieved documents
        retrieved_ids = [doc.metadata.get('doc_id') for doc, score in retrieved_docs_with_scores]
        
        # Check if the ground truth ID is in the retrieved IDs
        if ground_truth_id in retrieved_ids:
            hits += 1
            # Find the rank (position) of the correct document. Ranks are 1-based.
            rank = retrieved_ids.index(ground_truth_id) + 1
            reciprocal_ranks.append(1 / rank)
        else:
            reciprocal_ranks.append(0)
            misses.append({
                "question": question,
                "expected": ground_truth_id,
                "retrieved": retrieved_ids
            })

    total_questions = len(eval_dataset)
    hit_rate = (hits / total_questions) * 100
    mrr = np.mean(reciprocal_ranks)

    return {
        "hit_rate_at_k": k,
        "hit_rate": f"{hit_rate:.2f}%",
        "mrr": f"{mrr:.4f}",
        "total_questions": total_questions,
        "hits": hits,
        "misses_count": len(misses),
        "misses": misses
    }

In [15]:
evaluate_vectorstore_as_retriever(evaluation_dataset_AIgenerated, vectorStore, k=5)

Starting evaluation for k=5...


{'hit_rate_at_k': 5,
 'hit_rate': '85.00%',
 'mrr': '0.6917',
 'total_questions': 20,
 'hits': 17,
 'misses_count': 3,
 'misses': [{'question': '¿Cuál es la condición de convergencia para el método de la potencia directa?',
   'expected': 'chunk_3',
   'retrieved': ['chunk_5', 'chunk_17', 'chunk_27', 'chunk_26', 'chunk_13']},
  {'question': '¿Cuál es el pseudocodigo del método de la potencia?',
   'expected': 'chunk_4',
   'retrieved': ['chunk_19', 'chunk_30', 'chunk_5', 'chunk_12', 'chunk_17']},
  {'question': '¿Qué es la deflación y para qué se utiliza en el cálculo de autovectores?',
   'expected': 'chunk_6',
   'retrieved': ['chunk_3', 'chunk_5', 'chunk_0', 'chunk_2', 'chunk_4']}]}

In [16]:
evaluate_vectorstore_as_retriever(evaluation_dataset_AIgenerated, vectorStore, k=3)

Starting evaluation for k=3...


{'hit_rate_at_k': 3,
 'hit_rate': '75.00%',
 'mrr': '0.6667',
 'total_questions': 20,
 'hits': 15,
 'misses_count': 5,
 'misses': [{'question': '¿Cuál es la condición de convergencia para el método de la potencia directa?',
   'expected': 'chunk_3',
   'retrieved': ['chunk_5', 'chunk_17', 'chunk_27']},
  {'question': '¿Cuál es el pseudocodigo del método de la potencia?',
   'expected': 'chunk_4',
   'retrieved': ['chunk_19', 'chunk_30', 'chunk_5']},
  {'question': '¿Qué es la deflación y para qué se utiliza en el cálculo de autovectores?',
   'expected': 'chunk_6',
   'retrieved': ['chunk_3', 'chunk_5', 'chunk_0']},
  {'question': '¿Cuál es la diferencia fundamental entre la cuadratura de Newton-Cotes y la de Gauss-Legendre?',
   'expected': 'chunk_11',
   'retrieved': ['chunk_21', 'chunk_25', 'chunk_14']},
  {'question': 'Describe la regla de cuadratura de Gauss de dos puntos, incluyendo los valores de las abscisas y los coeficientes.',
   'expected': 'chunk_26',
   'retrieved': ['chu

### Testing de Re-ranker

In [23]:
# Creamos el retriever base. Este será "envuelto" por el compresor.
# Le damos un 'k' más alto porque esperamos que el filtro descarte algunos resultados.
base_retriever = vectorStore.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.1,"k": 10}
    #search_type="similarity", # 'similarity' with a 'k' is often more reliable
    #search_kwargs={"k": 10} # Retrieve more documents to give the reranker more to work with
)
# ¡IMPORTANTE! Usamos la MISMA instancia 'embedding_encoder' que para la DB.
redundant_filter = EmbeddingsRedundantFilter(embeddings=embedding_encoder)
reranker = CrossEncoderReranker(model=retrieval_reranker, top_n=3)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[redundant_filter, reranker]
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, 
    base_retriever=base_retriever
)

In [30]:
def evaluate_retriever(
    retriever: Any,
    eval_dataset: List[Dict[str, str]],
    retriever_name: str
) -> Dict[str, Any]:
    """
    Evaluates the performance of any retriever with a .invoke() method.

    Args:
        retriever (Any): A retriever object that has an .invoke(query) method
                         which returns a list of Document objects.
        eval_dataset (list): A list of dictionaries with "question" and 
                             "ground_truth_doc_id".
        retriever_name (str): A descriptive name for the retriever being tested
                              (e.g., "Base Retriever k=5").

    Returns:
        dict: A dictionary containing the calculated metrics and a list of misses.
    """
    hits = 0
    reciprocal_ranks = []
    misses = []  # To store information about failed queries for analysis

    print(f"--- Starting evaluation for: {retriever_name} ---")

    for item in eval_dataset:
        question = item["question"]
        ground_truth_id = item["ground_truth_doc_id"]
        
        # 1. Use the generic .invoke() method
        retrieved_docs = retriever.invoke(question)
        
        # 2. Extract doc_ids from the list of Document objects
        retrieved_ids = [doc.metadata.get('doc_id') for doc in retrieved_docs]
        
        # 3. Perform the evaluation logic (this part remains the same)
        if ground_truth_id in retrieved_ids:
            hits += 1
            # Ranks are 1-based
            rank = retrieved_ids.index(ground_truth_id) + 1
            reciprocal_ranks.append(1 / rank)
        else:
            reciprocal_ranks.append(0)
            misses.append({
                "question": question,
                "expected": ground_truth_id,
                "retrieved": retrieved_ids
            })

    total_questions = len(eval_dataset)
    hit_rate = (hits / total_questions) * 100
    mrr = np.mean(reciprocal_ranks) if reciprocal_ranks else 0

    print(f"Evaluation finished for: {retriever_name}\n")

    return {
        "retriever_name": retriever_name,
        "hit_rate": f"{hit_rate:.2f}%",
        "mrr": f"{mrr:.4f}",
        "total_questions": total_questions,
        "hits": hits,
        "misses_count": len(misses),
        "misses": misses
    }

def print_results(results: Dict[str, Any]):
    """Helper function to print evaluation results in a readable format."""
    print(f"--- Retrieval Evaluation Results for: {results['retriever_name']} ---")
    print(f"Hit Rate: {results['hit_rate']}")
    print(f"Mean Reciprocal Rank (MRR): {results['mrr']}")
    print(f"Correctly Retrieved (Hits): {results['hits']} / {results['total_questions']}")
    print("-" * 50)

    if results['misses_count'] > 0:
        print(f"\nAnalysis of {results['misses_count']} Misses:")
        # Print details for the first 3 misses for brevity
        for i, miss in enumerate(results['misses'][:3]):
            print(f"\nMiss {i+1}:")
            print(f"  Question: '{miss['question']}'")
            print(f"  Expected Doc ID: {miss['expected']}")
            print(f"  Retrieved IDs:   {miss['retrieved']}")
        if results['misses_count'] > 3:
            print("\n(And more...)")
    print("\n")

In [ ]:
# Evaluate the base retriever
base_retriever_results = evaluate_retriever(
    retriever=base_retriever,
    eval_dataset=evaluation_dataset_AIgenerated,
    retriever_name="Base Retriever"
)
# Evaluate the compression retriever
compression_retriever_results = evaluate_retriever(
    retriever=compression_retriever,
    eval_dataset=evaluation_dataset_AIgenerated,
    retriever_name="Compression Retriever"
)

# --- Reporting Phase ---
print_results(base_retriever_results)
print_results(compression_retriever_results)

--- Starting evaluation for: Base Retriever ---
Evaluation finished for: Base Retriever

--- Starting evaluation for: Compression Retriever ---


In [36]:
print_results(base_retriever_results)

--- Retrieval Evaluation Results for: Base Retriever ---
Hit Rate: 95.00%
Mean Reciprocal Rank (MRR): 0.7083
Correctly Retrieved (Hits): 19 / 20
--------------------------------------------------

Analysis of 1 Misses:

Miss 1:
  Question: '¿Cuál es el pseudocodigo del método de la potencia?'
  Expected Doc ID: chunk_4
  Retrieved IDs:   ['chunk_19', 'chunk_30', 'chunk_5', 'chunk_12', 'chunk_17', 'chunk_21', 'chunk_2', 'chunk_14', 'chunk_28', 'chunk_27']




## Setup Fase Generativa

In [21]:
def LoadGoogleLLM():
    """Instantiates and returns a Google LLM through the LangChain interface."""
    # Ensure your GOOGLE_API_KEY is set as an environment variable
    llm = ChatGoogleGenerativeAI(model="gemma-3-27b-it", temperature=0)
    return llm

def DefineGemmaPrompt():
    """
    Defines the prompt template for Gemma 3 models via Google API.
    
    Since Gemma 3 does not support system prompts through this API, this function
    combines the system instructions and the user query into a single human/user
    message template.
    """
    system_instructions = """Eres un experto en pedagogía para estudiantes universitarios de la generación Z y profesor de la cátedra de Métodos Numéricos en la Facultad de Ingeniería. Tu objetivo es guiar al usuario a lograr una comprensión más profunda sobre su pregunta.

Recibirás una PREGUNTA y un CONTEXTO de las notas de clase. Sigue estas reglas estrictamente:
1. Responde a la PREGUNTA utilizando ÚNICAMENTE el CONTEXTO proporcionado. No uses información de otras fuentes. Si no hay CONTEXTO, indica que la respuesta no ha sido encontrada.
2. Formatea siempre tus respuestas utilizando Markdown para mejorar la legibilidad.
3. Después de tu explicación, incluye una sección de TAREAS ACCIONABLES o PREGUNTAS DE REFLEXIÓN para que el estudiante aplique o profundice su conocimiento.
4. Responde siempre en español. Sé útil y claro."""

    # Combine the instructions and the dynamic parts into a single template string.
    # The model will treat the entire block as the user's input.
    full_prompt_string = (
        f"{system_instructions}\n\n"
        "--- \n\n"  # Using a separator can sometimes help the model distinguish instructions from data.
        "CONTEXTO:\n{context}\n\n"
        "PREGUNTA:\n{question}"
    )

    # Create the template from a single string. LangChain will treat this
    # as a single "human" message by default in many chains.
    prompt_template = ChatPromptTemplate.from_template(full_prompt_string)
    
    return prompt_template

def ProcessInput(question,retriever,llm):
    #Data pipeline: user query->retrieve chunks->join them->inject in prompt-> get LLM response
    prompt = DefineGemmaPrompt()
    rag_chain = (
        {"context": retriever | join_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain.invoke(question)

In [19]:
#llm=LoadOllamaLLM()
llm=LoadGoogleLLM()

In [24]:
#Uso de herramienta
query="Que pasa si ingreso un autovector como vector inicial en el metodo de la potencia?"

response=ProcessInput(query,base_retriever,llm)
print(response)

De acuerdo al contexto proporcionado, si ingresas un autovector como vector inicial en el método de la potencia, el proceso se simplifica significativamente. 

El método de la potencia, como se explica, converge al autovector dominante (el asociado al autovalor de mayor valor absoluto) a través de sucesivas premultiplicaciones de un vector inicial por la matriz `A`.  Si el vector inicial `x` ya es un autovector `v_1` asociado al autovalor dominante `λ_1`, entonces:

`A x = A v_1 = λ_1 v_1 = λ_1 x`

Cada iteración del método simplemente multiplicará el vector `x` por el autovalor `λ_1`.  El vector permanecerá en la dirección del autovector `v_1` y el cociente `x_{k+1}(j) / x_k(j)` convergerá inmediatamente a `λ_1` sin necesidad de múltiples iteraciones.  El escalamiento (normalización) seguirá siendo necesario para evitar problemas de overflow o underflow, pero la convergencia será mucho más rápida y directa.

---

**TAREAS ACCIONABLES / PREGUNTAS DE REFLEXIÓN:**

1.  **Considera una ma